In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from downcast import reduce
import seaborn as sns
from sklearn.model_selection import train_test_split
import random
from typing import Union
from tqdm.notebook import tqdm_notebook as tqdm
import gc
from sklearn.preprocessing import LabelEncoder

##  Preparing data
#### Note:
Throughout this notebook memory is a concern. A pretty big one too. I've had the kernel die multiple times due to insufficient memory, and del and gc.collect() don't really seem to do the trick. This is probably due to OS caching. For this reason we only use a part of our sales dataset (admittedly a pretty large one), as well as using .pkl for saving/reading data at crucial points. csv's just take up too much space. It is likely the kernel will die when running this, especially during the roll mean calculation. If it does just start off from there.

In [2]:
calendar = pd.read_csv('input/calendar.csv', parse_dates=['date'])
sell_prices = pd.read_csv('input/sell_prices.csv')
sales = pd.read_csv('input/sales_train_evaluation.csv')
sales_valid = pd.read_csv('input/sales_train_validation.csv')

In [3]:
#Due to large filesizes we will use downcast to reduce filesize without impacting provided data
calendar = reduce(calendar)
sell_prices = reduce(sell_prices)
sales = reduce(sales)
sales_valid = reduce(sales_valid)

In [4]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [5]:
sales

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [6]:
# We need to add empty columns for our submission
for d in range(1942, 1970):
    sales['d_%d' % d] = 0
sales

# I get a DF fragmented warning, not really sure how to go around it and honestly can't really be bothered
# We will delete sales later on to preserve memory anyway (speaking of, 16 gigs is starting to feel small)

sales_df = sales.copy()

/tmp/ipykernel_14751/3469712393.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  sales['d_%d' % d] = 0


In [7]:
# Haha I wish this worked like I hoped it did
del sales
gc.collect()
sell_prices

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.000000
6841117,WI_3,FOODS_3_827,11618,1.000000
6841118,WI_3,FOODS_3_827,11619,1.000000
6841119,WI_3,FOODS_3_827,11620,1.000000


## Merging the dataframes
In order to create a matrix that could work as our input we will need to merge features from all 3 dataframes. The only shared information between _calendar_ and _sales_ is _d_ or the day identifier. However, the _d_ columns in _sales_ create a 2-dimensional matrix which we will need to convert to a single vector. This will add a lot of new rows. I'd say a million but its way more than that. This might be an issue later. Still, once _d_ is a column we will be able to merge with _calendar_. After that we will have shared column for every column in _sell_pices_ with the exception of well... the actual price. This will make for an easy merge.

In [8]:
# melting our sales to make d a single column to prepare for merging with other datasets
sales_d = pd.melt(
    sales_df,
    id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'],
    var_name='d',
    value_name='items_sold')

In [9]:
sales_d

,id,item_id,dept_id,cat_id,store_id,state_id,d,items_sold
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...,...
60034805,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1969,0
60034806,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1969,0
60034807,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1969,0
60034808,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1969,0


In [10]:
sales_d = sales_d.merge(calendar, on='d', how='left')
sales_d

,id,item_id,dept_id,cat_id,store_id,state_id,d,items_sold,date,wm_yr_wk,...,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60034805,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,11621,...,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0
60034806,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,11621,...,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0
60034807,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,11621,...,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0
60034808,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,11621,...,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0


In [11]:
sell_prices

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.000000
6841117,WI_3,FOODS_3_827,11618,1.000000
6841118,WI_3,FOODS_3_827,11619,1.000000
6841119,WI_3,FOODS_3_827,11620,1.000000


In [12]:
sales_d = sales_d.merge(sell_prices, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')

In [13]:
# Actually lets just take a part of this, 2011+ is just far too much for my PC to handle
sales_d = sales_d[sales_d['date']>'2014-03-19']

In [14]:
# Let us rearrange our dataframe into something a bit more logical
sales_d = sales_d[['id', 'date', 'item_id','dept_id','cat_id','store_id','state_id','wm_yr_wk','weekday','wday','month','year',
  'event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI','items_sold','sell_price']]
sales_d

,id,date,item_id,dept_id,cat_id,store_id,state_id,wm_yr_wk,weekday,wday,...,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,items_sold,sell_price
34941540,HOBBIES_1_001_CA_1_evaluation,2014-03-20,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,11407,Thursday,6,...,2014,NaN,NaN,NaN,NaN,0,0,0,1,8.257812
34941541,HOBBIES_1_002_CA_1_evaluation,2014-03-20,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,11407,Thursday,6,...,2014,NaN,NaN,NaN,NaN,0,0,0,0,3.970703
34941542,HOBBIES_1_003_CA_1_evaluation,2014-03-20,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,11407,Thursday,6,...,2014,NaN,NaN,NaN,NaN,0,0,0,0,2.970703
34941543,HOBBIES_1_004_CA_1_evaluation,2014-03-20,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,11407,Thursday,6,...,2014,NaN,NaN,NaN,NaN,0,0,0,3,4.640625
34941544,HOBBIES_1_005_CA_1_evaluation,2014-03-20,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,11407,Thursday,6,...,2014,NaN,NaN,NaN,NaN,0,0,0,0,3.080078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60034805,FOODS_3_823_WI_3_evaluation,2016-06-19,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,11621,Sunday,2,...,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,0,2.980469
60034806,FOODS_3_824_WI_3_evaluation,2016-06-19,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,11621,Sunday,2,...,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,0,2.480469
60034807,FOODS_3_825_WI_3_evaluation,2016-06-19,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,11621,Sunday,2,...,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,0,3.980469
60034808,FOODS_3_826_WI_3_evaluation,2016-06-19,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,11621,Sunday,2,...,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,0,1.280273


In [15]:
# At this point let's delete the old csv's, not that it will do us any good
del calendar, sell_prices, sales_df
gc.collect()

0

## Removing and adding features

In [16]:
# Creating a numerical feature for the presence of events
sales_d['event_num'] = (sales_d['event_name_1'].notna()).astype(int) + (sales_d['event_name_2'].notna()).astype(int)

In [17]:
sales_d

,id,date,item_id,dept_id,cat_id,store_id,state_id,wm_yr_wk,weekday,wday,...,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,items_sold,sell_price,event_num
34941540,HOBBIES_1_001_CA_1_evaluation,2014-03-20,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,11407,Thursday,6,...,NaN,NaN,NaN,NaN,0,0,0,1,8.257812,0
34941541,HOBBIES_1_002_CA_1_evaluation,2014-03-20,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,11407,Thursday,6,...,NaN,NaN,NaN,NaN,0,0,0,0,3.970703,0
34941542,HOBBIES_1_003_CA_1_evaluation,2014-03-20,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,11407,Thursday,6,...,NaN,NaN,NaN,NaN,0,0,0,0,2.970703,0
34941543,HOBBIES_1_004_CA_1_evaluation,2014-03-20,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,11407,Thursday,6,...,NaN,NaN,NaN,NaN,0,0,0,3,4.640625,0
34941544,HOBBIES_1_005_CA_1_evaluation,2014-03-20,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,11407,Thursday,6,...,NaN,NaN,NaN,NaN,0,0,0,0,3.080078,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60034805,FOODS_3_823_WI_3_evaluation,2016-06-19,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,11621,Sunday,2,...,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,0,2.980469,2
60034806,FOODS_3_824_WI_3_evaluation,2016-06-19,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,11621,Sunday,2,...,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,0,2.480469,2
60034807,FOODS_3_825_WI_3_evaluation,2016-06-19,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,11621,Sunday,2,...,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,0,3.980469,2
60034808,FOODS_3_826_WI_3_evaluation,2016-06-19,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,11621,Sunday,2,...,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,0,1.280273,2


In [18]:
# We don't need the snap features to differ by state, so lets combine them into a single feature
sales_d.loc[sales_d['state_id'] == 'CA', 'snap'] = sales_d.loc[sales_d['state_id'] == 'CA']['snap_CA']
sales_d.loc[sales_d['state_id'] == 'TX', 'snap'] = sales_d.loc[sales_d['state_id'] == 'TX']['snap_TX']
sales_d.loc[sales_d['state_id'] == 'WI', 'snap'] = sales_d.loc[sales_d['state_id'] == 'WI']['snap_WI']
sales_d['snap'] = sales_d['snap'].astype(int)

In [19]:
# We can now remove the aggregated columns as well as weekday, since that data is already provided by wday
sales_d.drop(['event_name_1', 'event_name_2', 'event_type_1','event_type_2','weekday', 
              'snap_CA', 'snap_TX', 'snap_WI'],axis=1,inplace=True)

In [20]:
# As we observed sales always went up on weekends, let us add a feature that identifies Saturdays and Sundays for us
sales_d['weekend'] = (sales_d['wday'] <= 2).astype(int)

# There was an additional seasonal trend that we observed, lets add a feature that identifies each season
# on a scale from 0 to 3, winter (Dec-Feb) and autumn (Sep-Nov) respectively.
sales_d['season'] = (sales_d['month'] / 3).astype(int)
sales_d['season'] = (sales_d['season'] % 4).astype(int) # forgot that 12/3 = 4


## Categorical Features encoding

Lets encode our categorical features to a machine learning friendly format. We use label encoding instead of one hot encoding due to memory concerns (this is a reccuring theme with this dataset).

In [21]:
c_features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

for c in c_features:
    le = LabelEncoder()
    sales_d[c+'_label'] = le.fit_transform(sales_d[c])

## Lags and Rolling Mean
Due to memory concerns we do some saving and loading before and after this section. This is so that we don't lose our progress in case the kernel dies due to insufficient memory. Personally I had to rerun the notebook from this point, but for PCs with higher ram amounts this shouldnt be an issue.

In [23]:
sales_d.to_pickle('sales_d_raw.pkl')

In [3]:
sales_d = pd.read_pickle('sales_d_raw.pkl')

In [4]:
sales_d = reduce(sales_d)

In [5]:
# lags
lags = [7, 28, 35, 42, 60, 360]
lag_columns = [f'lag_{lag}' for lag in lags] # name columns
for lag, lag_column in zip(lags, lag_columns):
    sales_d[lag_column] = sales_d[['id',"items_sold"]].groupby('id')['items_sold'].shift(lag)


In [6]:
# rolling mean
rolls = [7, 28, 35, 42, 60, 360]
for r in rolls:
    for lag, lag_column in zip(lags, lag_columns):
        sales_d[f'rmean_{lag}_{r}'] = sales_d[['id', lag_column]].groupby('id')[lag_column].transform(lambda x: x.rolling(r).mean())


In [7]:
sales_d.to_pickle('sales_lags_rolls.pkl')

In [8]:
sales_final = pd.read_pickle('sales_lags_rolls.pkl')
sales_final = reduce(sales_final)
sales_final

,id,date,item_id,dept_id,cat_id,store_id,state_id,wm_yr_wk,wday,month,...,rmean_35_60,rmean_42_60,rmean_60_60,rmean_360_60,rmean_7_360,rmean_28_360,rmean_35_360,rmean_42_360,rmean_60_360,rmean_360_360
34941540,HOBBIES_1_001_CA_1_evaluation,2014-03-20,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,11407,6,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34941541,HOBBIES_1_002_CA_1_evaluation,2014-03-20,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,11407,6,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34941542,HOBBIES_1_003_CA_1_evaluation,2014-03-20,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,11407,6,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34941543,HOBBIES_1_004_CA_1_evaluation,2014-03-20,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,11407,6,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34941544,HOBBIES_1_005_CA_1_evaluation,2014-03-20,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,11407,6,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60034805,FOODS_3_823_WI_3_evaluation,2016-06-19,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,11621,2,6,...,0.333252,0.250000,0.216675,0.333252,0.585938,0.619629,0.611328,0.602539,0.583496,0.372314
60034806,FOODS_3_824_WI_3_evaluation,2016-06-19,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,11621,2,6,...,0.250000,0.250000,0.133301,0.000000,0.047211,0.047211,0.041656,0.041656,0.022217,0.172241
60034807,FOODS_3_825_WI_3_evaluation,2016-06-19,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,11621,2,6,...,0.816895,0.850098,1.033203,0.916504,0.549805,0.608398,0.605469,0.605469,0.638672,1.000000
60034808,FOODS_3_826_WI_3_evaluation,2016-06-19,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,11621,2,6,...,1.083008,1.099609,0.899902,0.000000,1.200195,1.200195,1.164062,1.144531,1.078125,0.163940


## Train/Test split
Now lets split up our dataset into train/validation and testtest. Given that we're meant to predict 
28 days specifically, instead of using traintestsplit we will just split this manually.
28 days for submission, 28 days for test, rest for train.
Values calculated manually from d_ values

In [9]:
df_train = sales_final[sales_final['date'] < '2016-04-25']
df_test = sales_final[sales_final['date'] >= '2016-04-25']
df_test = df_test[df_test['date'] <= '2016-05-22']
df_sub = sales_final[sales_final['date'] > '2016-05-22']

In [ ]:
# This really does nothing as far as I can tell, sure it tells python that this part of memory is free to use,
# which I suppose should suffice, but my OS keeps that data cached anyway, thus not freeing any RAM.
# One solution would be to flush the OS cache somehow, but I don't have the time to wonder about that right now.
# Never thought I'd say this but, I miss C++
del sales_final
gc.collect()

One last sanity check before saving each dataset

In [10]:
df_train

,id,date,item_id,dept_id,cat_id,store_id,state_id,wm_yr_wk,wday,month,...,rmean_35_60,rmean_42_60,rmean_60_60,rmean_360_60,rmean_7_360,rmean_28_360,rmean_35_360,rmean_42_360,rmean_60_360,rmean_360_360
34941540,HOBBIES_1_001_CA_1_evaluation,2014-03-20,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,11407,6,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34941541,HOBBIES_1_002_CA_1_evaluation,2014-03-20,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,11407,6,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34941542,HOBBIES_1_003_CA_1_evaluation,2014-03-20,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,11407,6,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34941543,HOBBIES_1_004_CA_1_evaluation,2014-03-20,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,11407,6,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34941544,HOBBIES_1_005_CA_1_evaluation,2014-03-20,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,11407,6,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58327365,FOODS_3_823_WI_3_evaluation,2016-04-24,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,11613,2,4,...,0.600098,0.633301,0.783203,0.533203,0.588867,0.600098,0.608398,0.630371,0.636230,0.316650
58327366,FOODS_3_824_WI_3_evaluation,2016-04-24,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,11613,2,4,...,0.000000,0.000000,0.000000,0.199951,0.022217,0.033325,0.033325,0.033325,0.033325,0.252686
58327367,FOODS_3_825_WI_3_evaluation,2016-04-24,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,11613,2,4,...,0.883301,0.933105,0.733398,1.066406,0.636230,0.652832,0.655762,0.652832,0.633301,1.000000
58327368,FOODS_3_826_WI_3_evaluation,2016-04-24,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,11613,2,4,...,1.000000,1.066406,1.233398,0.066650,1.075195,1.025391,0.994629,0.972168,0.938965,0.316650


In [11]:
df_test

,id,date,item_id,dept_id,cat_id,store_id,state_id,wm_yr_wk,wday,month,...,rmean_35_60,rmean_42_60,rmean_60_60,rmean_360_60,rmean_7_360,rmean_28_360,rmean_35_360,rmean_42_360,rmean_60_360,rmean_360_360
58327370,HOBBIES_1_001_CA_1_evaluation,2016-04-25,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,11613,3,4,...,0.916504,0.816895,0.850098,0.733398,0.674805,0.664062,0.658203,0.649902,0.633301,0.616699
58327371,HOBBIES_1_002_CA_1_evaluation,2016-04-25,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,11613,3,4,...,0.183350,0.216675,0.150024,0.116638,0.355469,0.358398,0.347168,0.350098,0.344482,0.316650
58327372,HOBBIES_1_003_CA_1_evaluation,2016-04-25,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,11613,3,4,...,0.383301,0.366699,0.333252,0.199951,0.527832,0.505371,0.502930,0.505371,0.486084,0.172241
58327373,HOBBIES_1_004_CA_1_evaluation,2016-04-25,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,11613,3,4,...,2.050781,2.050781,2.017578,2.349609,1.958008,2.021484,2.001953,2.019531,2.044922,2.396484
58327374,HOBBIES_1_005_CA_1_evaluation,2016-04-25,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,11613,3,4,...,1.133789,1.016602,0.866699,1.233398,1.116211,1.122070,1.116211,1.114258,1.130859,1.063477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59181085,FOODS_3_823_WI_3_evaluation,2016-05-22,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,11617,2,5,...,0.266602,0.250000,0.566895,0.316650,0.611328,0.588867,0.588867,0.583496,0.605469,0.336182
59181086,FOODS_3_824_WI_3_evaluation,2016-05-22,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,11617,2,5,...,0.133301,0.099976,0.000000,0.199951,0.041656,0.024994,0.022217,0.016663,0.033325,0.213867
59181087,FOODS_3_825_WI_3_evaluation,2016-05-22,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,11617,2,5,...,1.000000,1.000000,0.899902,1.133789,0.605469,0.627930,0.636230,0.636230,0.658203,1.016602
59181088,FOODS_3_826_WI_3_evaluation,2016-05-22,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,11617,2,5,...,0.933105,1.033203,0.983398,0.000000,1.164062,1.097656,1.075195,1.061523,1.002930,0.244385


In [12]:
df_sub

,id,date,item_id,dept_id,cat_id,store_id,state_id,wm_yr_wk,wday,month,...,rmean_35_60,rmean_42_60,rmean_60_60,rmean_360_60,rmean_7_360,rmean_28_360,rmean_35_360,rmean_42_360,rmean_60_360,rmean_360_360
59181090,HOBBIES_1_001_CA_1_evaluation,2016-05-23,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,11617,3,5,...,1.049805,1.049805,0.966797,0.383301,0.747070,0.683105,0.674805,0.672363,0.655762,0.600098
59181091,HOBBIES_1_002_CA_1_evaluation,2016-05-23,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,11617,3,5,...,0.199951,0.216675,0.199951,0.216675,0.347168,0.355469,0.355469,0.355469,0.350098,0.313965
59181092,HOBBIES_1_003_CA_1_evaluation,2016-05-23,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,11617,3,5,...,0.466553,0.366699,0.383301,0.250000,0.561035,0.539062,0.527832,0.505371,0.502930,0.177734
59181093,HOBBIES_1_004_CA_1_evaluation,2016-05-23,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,11617,3,5,...,1.900391,1.816406,2.017578,2.482422,1.914062,1.966797,1.958008,1.983398,2.007812,2.406250
59181094,HOBBIES_1_005_CA_1_evaluation,2016-05-23,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,11617,3,5,...,1.099609,1.200195,1.150391,0.883301,1.191406,1.127930,1.116211,1.122070,1.125000,1.041992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60034805,FOODS_3_823_WI_3_evaluation,2016-06-19,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,11621,2,6,...,0.333252,0.250000,0.216675,0.333252,0.585938,0.619629,0.611328,0.602539,0.583496,0.372314
60034806,FOODS_3_824_WI_3_evaluation,2016-06-19,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,11621,2,6,...,0.250000,0.250000,0.133301,0.000000,0.047211,0.047211,0.041656,0.041656,0.022217,0.172241
60034807,FOODS_3_825_WI_3_evaluation,2016-06-19,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,11621,2,6,...,0.816895,0.850098,1.033203,0.916504,0.549805,0.608398,0.605469,0.605469,0.638672,1.000000
60034808,FOODS_3_826_WI_3_evaluation,2016-06-19,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,11621,2,6,...,1.083008,1.099609,0.899902,0.000000,1.200195,1.200195,1.164062,1.144531,1.078125,0.163940


In [13]:
df_train.to_pickle('df_train_final.pkl')

In [14]:
df_test.to_pickle('df_test_final.pkl')

In [15]:
df_sub.to_pickle('df_test_sub.pkl')